In [1]:
import sentencepiece as spm


trained_sp = spm.SentencePieceProcessor()
trained_sp.load("models/trained_spiece.model")

True

In [2]:
sample_text = "Từ giữa năm ngoái, một số gói thầu của dự án triển khai trở lại. Tổng công ty Đầu tư phát triển đường cao tốc Việt Nam (VEC - chủ đầu tư) đặt mục tiêu thông xe kỹ thuật các đoạn sử dụng vốn của Ngân hàng Phát triển châu Á (ADB) trước tháng 10 năm nay."


encoded_pieces = trained_sp.encode(sample_text, out_type=str)
encoded_ids = trained_sp.encode(sample_text)


print("Encoded pieces:", encoded_pieces)
print("Encoded IDs:", encoded_ids)

Encoded pieces: ['▁Từ', '▁giữa', '▁năm', '▁ngoái', ',', '▁một', '▁số', '▁gói', '▁thầu', '▁của', '▁dự', '▁án', '▁triển', '▁khai', '▁trở', '▁lại', '.', '▁Tổng', '▁công', '▁ty', '▁Đầu', '▁tư', '▁phát', '▁triển', '▁đường', '▁cao', '▁tốc', '▁Việt', '▁Nam', '▁(', 'VEC', '▁-', '▁chủ', '▁đầu', '▁tư', ')', '▁đặt', '▁mục', '▁tiêu', '▁thông', '▁xe', '▁kỹ', '▁thuật', '▁các', '▁đoạn', '▁sử', '▁dụng', '▁vốn', '▁của', '▁Ngân', '▁hàng', '▁Phát', '▁triển', '▁châu', '▁Á', '▁(', 'ADB', ')', '▁trước', '▁tháng', '▁10', '▁năm', '▁nay', '.']
Encoded IDs: [709, 432, 38, 1419, 3, 18, 44, 1378, 1504, 5, 142, 148, 217, 260, 237, 68, 15, 360, 17, 216, 1468, 174, 103, 217, 163, 106, 815, 75, 55, 27, 14015, 36, 186, 48, 174, 45, 555, 540, 277, 65, 118, 714, 544, 7, 505, 205, 109, 653, 5, 1033, 81, 964, 217, 1231, 809, 27, 20787, 45, 110, 131, 363, 38, 173, 15]


In [3]:
decoded_text = trained_sp.decode(encoded_ids)


print("Decoded text:", decoded_text)


Decoded text: Từ giữa năm ngoái, một số gói thầu của dự án triển khai trở lại. Tổng công ty Đầu tư phát triển đường cao tốc Việt Nam (VEC - chủ đầu tư) đặt mục tiêu thông xe kỹ thuật các đoạn sử dụng vốn của Ngân hàng Phát triển châu Á (ADB) trước tháng 10 năm nay.


In [9]:
import pandas as pd

In [10]:
import torch
import pprint
import evaluate
import numpy as np
 
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)


In [11]:
dataset = pd.read_csv('data/processed/processed_data.csv')


In [12]:
dataset1 = dataset.head(20)

In [13]:
from datasets import Dataset
dataset1 = Dataset.from_pandas(dataset1)

In [14]:
dataset1

Dataset({
    features: ['Summary', 'Text'],
    num_rows: 20
})

In [16]:
full_dataset = dataset1.train_test_split(test_size=0.2, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['Summary', 'Text'],
    num_rows: 16
})
Dataset({
    features: ['Summary', 'Text'],
    num_rows: 4
})


In [21]:
MODEL = 't5-small'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 5
OUT_DIR = './result666'
MAX_LENGTH = 512 

In [18]:
tokenizer = T5Tokenizer('models/trained_spiece.model')

In [19]:
text = "Đây là một ví dụ về cách sử dụng tokenizer T5 cho tiếng Việt."

# Tokenize the text
tokens = tokenizer.tokenize(text)

print(tokens)

['▁Đây', '▁là', '▁một', '▁ví', '▁dụ', '▁về', '▁cách', '▁sử', '▁dụng', '▁token', 'i', 'zer', '▁T', '5', '▁cho', '▁tiếng', '▁Việt', '.']


In [23]:
from functools import partial
from transformers import T5Tokenizer

# Initialize the tokenizer
# tokenizer = T5Tokenizer.from_pretrained(MODEL)
tokenizer = T5Tokenizer('models/trained_spiece.model')
# Function to convert text data into model inputs and targets
def preprocess_function(examples, tokenizer):
    inputs = [f"summarize: {text}" for text in examples['Text']]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summary']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Create a partial function with the tokenizer
preprocess_function_with_tokenizer = partial(preprocess_function, tokenizer=tokenizer)

# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function_with_tokenizer,
    batched=True,
    num_proc=NUM_PROCS
)


Map (num_proc=4):   0%|          | 0/16 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4 [00:00<?, ? examples/s]

# cm


In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)
 
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"summarize: {text}" for text in examples['Text']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )
 
    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summary']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
 
# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

# tuyet

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [24]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

60,506,624 total parameters.
60,506,624 training parameters.


In [30]:
rouge = evaluate.load("rouge")
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [26]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [28]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)
 
history = trainer.train()

  0%|          | 0/20 [00:00<?, ?it/s]

{'loss': 17.3274, 'grad_norm': 140.64414978027344, 'learning_rate': 2.0000000000000003e-06, 'epoch': 2.5}
{'loss': 16.5515, 'grad_norm': 113.92288208007812, 'learning_rate': 4.000000000000001e-06, 'epoch': 5.0}
{'train_runtime': 70.1472, 'train_samples_per_second': 1.14, 'train_steps_per_second': 0.285, 'train_loss': 16.93944320678711, 'epoch': 5.0}


In [50]:
trainer.save_model("joemama")

In [33]:
model_path = f"{OUT_DIR}/checkpoint-20"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer('models/trained_spiece.model')

In [47]:
testdata = "summarize: " + dataset1['Text'][5]
summary = dataset1['Summary'][5]
print(testdata)

summarize: Lý Quang Diệu được cho là người đứng đằng sau "điều kỳ diệu Singapore" Người phát ngôn của ông Lý, Yeong Yoon Ying , nói ông hiện đang được chữa trị tại bệnh viện đa khoa Singapore sau khi phải vào viện vì sốt cao và ho, nghi do nhiễm trùng. "Ông Lý hiện đang được điều trị bằng thuốc kháng sinh và đang hồi phục. Ông được các bác sỹ khuyên không nên xuất hiện trước công chúng." Ông Lý Quang Diệu năm nay 90 tuổi và vẫn giữ vai trò dân biểu tuy đã thôi chức cố vấn chính phủ từ năm 2011. Ông sẽ không ăn tối mừng Năm mới Âm lịch với đại diện cử tri tại khu vực của ông, lần thứ hai trong hai năm. Lý Quang Diệu, cha đẻ của đương kim thủ tướng Lý Hiển Long, được xem như người có công đưa Singapore trở thành một trong những nền kinh tế hùng mạnh nhất Á châu. Ông làm thủ tướng từ 1959, khi Singapore giành độc lập từ Anh quốc, cho tới năm 1990. Kế nhiệm ông là Goh Chok Tong, và từ 2004 là Lý Hiển Long, con trai ông. Đảng Nhân dân Hành động của ông Lý nắm quyền từ 1959 t

In [48]:
print(summary)

Chính phủ Singapore cho hay cựu thủ tướng, người được cho là sáng lập gia của nhà nước Singapore, ông Lý Quang Diệu vừa phải nhập viện hôm Chủ nhật 2/2.


In [56]:
# from transformers import pipeline

# summarizer = pipeline("summarization", model=f"{OUT_DIR}/checkpoint-16" )
# pred = summarizer(text)
# pred

OSError: Can't load tokenizer for './result666/checkpoint-16'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './result666/checkpoint-16' is the correct path to a directory containing all relevant files for a T5TokenizerFast tokenizer.

In [59]:
text = dataset1['Text'][5]

In [64]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=100,
        num_beams=num_beams,
        # early_stopping=True,
    )

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [65]:
print(text)

Lý Quang Diệu được cho là người đứng đằng sau "điều kỳ diệu Singapore" Người phát ngôn của ông Lý, Yeong Yoon Ying , nói ông hiện đang được chữa trị tại bệnh viện đa khoa Singapore sau khi phải vào viện vì sốt cao và ho, nghi do nhiễm trùng. "Ông Lý hiện đang được điều trị bằng thuốc kháng sinh và đang hồi phục. Ông được các bác sỹ khuyên không nên xuất hiện trước công chúng." Ông Lý Quang Diệu năm nay 90 tuổi và vẫn giữ vai trò dân biểu tuy đã thôi chức cố vấn chính phủ từ năm 2011. Ông sẽ không ăn tối mừng Năm mới Âm lịch với đại diện cử tri tại khu vực của ông, lần thứ hai trong hai năm. Lý Quang Diệu, cha đẻ của đương kim thủ tướng Lý Hiển Long, được xem như người có công đưa Singapore trở thành một trong những nền kinh tế hùng mạnh nhất Á châu. Ông làm thủ tướng từ 1959, khi Singapore giành độc lập từ Anh quốc, cho tới năm 1990. Kế nhiệm ông là Goh Chok Tong, và từ 2004 là Lý Hiển Long, con trai ông. Đảng Nhân dân Hành động của ông Lý nắm quyền từ 1959 tới nay, và 

In [66]:
summarize_text(text, model, tokenizer)

', ông Lý Quang Diệu nhiều lần cảnh báo về sự trỗi dậy của Trung Quốc cho đây là một thách thức ở Thái Bình Dương. Ông cũng đánh giá rằng chính sự hiện diện của ông Lý Quang Diệu nhiều lần cảnh báo về sự trỗi dậy của Trung Quốc cho đây là một thách thức ở Thái Bình Dương. Ông cũng đánh giá rằng chính sự hiện diện của ông Lý Quang Diệu nhiều lần cảnh báo về sự trỗi dậy của Trung Quốc cho đây là một thách thức ở'